#**Change in Earth's albedo and its dependence on land cover changes in the past 20 years**

**Content creators:** Oz Kira, Julius Bamah

**Content reviewers:** Yuhan Douglas Rao, Abigail Bodner

**Content editors:** Zane Mitrevica, Natalie Steinemann

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

In [ ]:
# @title #**Project background** 
#This will be a short video introducing the content creator(s) and motivating the research direction of the template.
#The Tech team will add code to format and display the video

The global radiative budget is affected by land cover. Regarding vegetation land cover (e.g., forests, grasslands, agricultural fields, etc.), vegetation sequesters carbon, which reduces the greenhouse effect but absorbs more radiation and reduces earth albedo, which counteracts carbon sequestration. In this project, we will evaluate the albedo change vs. carbon sequestration over the past 24 years. In addition, we will track significant land use changes, specifically agricultural land creation, and abandonment. 

**In this project**, you will be given the opportunity to explore terrestrial remote sensing and meteorological data from MODIS and MERRA-2 to extract reflectance, albedo, and meteorological variables and information on land cover change in your region of interest. We encourage you to use this data to investigate the relationships between these variables and how they affect the global radiative budget. In addition, you can track the phenomenon of agricultural land abandonment and map if the lands were abandoned due to climate change.

#**Project template**
<p align='center'><a href="https://github.com/ClimatematchAcademy/course-content/tree/main/projects/template_images/albedo_template_map.svg"><img src="https://github.com/ClimatematchAcademy/course-content/tree/main/projects/template_images/albedo_template_map.svg" alt="Change in Earth's albedo and its dependence on land cover changes in the past 20 years" vw="100" vh="75" /></a></p>

#**Data exploration notebook**
## Project setup

To use the script the following proceures must be ensured:
1. Register an account on register at NASA earth data portal: https://urs.earthdata.nasa.gov/users/new 
2.Download python from here : https://www.python.org/downloads/ (please note: if neccessary)
2. pip install the following libaries into yor Jupyter notebook 



Please run the following cells!
    



In [ ]:
!pip install cartopy
!pip install DateTime 
!pip install matplotlib
!pip install pyhdf
!pip install numpy
!pip install pandas
!pip install modis-tools
!pip install 


# the further information on the MODIS data can be found here : https://amanbagrecha.github.io/post/rs_gis/download-modis-data-using-cmr-api-in-python/
#Import the libraries
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler
from typing_extensions import Literal
from pyhdf.SD import SD, SDC
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 41.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for cartopy: filename=Cartopy-0.21.1-cp310-cp310-linux_x86_64.whl size=11102762 sha256=59051dcb4e9cd6032b499d1953b151bd631d2fbf16f1d08a2336a48cf125ccf1
  Stored in directory: /root/.cache/pip/wheels/30/b0/1a/1c1909e00c76653dc4e2ff48555257c0eb2d1698280c8d9955
Successfully built cartopy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

ModuleNotFoundError: ignored

#**MODIS (Moderate Resolution Imaging Spectroradiometer)**

MODIS (Moderate Resolution Imaging Spectroradiometer) is a key instrument aboard the Terra (originally known as EOS AM-1) and Aqua (originally known as EOS PM-1) satellites. Terra's orbit around the Earth is timed so that it passes from north to south across the equator in the morning, while Aqua passes south to north over the equator in the afternoon. Terra MODIS and Aqua MODIS are viewing the entire Earth's surface every 1 to 2 days, acquiring data in 36 spectral bands, or groups of wavelengths (see MODIS Technical Specifications). These data will improve our understanding of global dynamics and processes occurring on the land, in the oceans, and in the lower atmosphere. MODIS is playing a vital role in the development of validated, global, interactive Earth system models able to predict global change accurately enough to assist policy makers in making sound decisions concerning the protection of our environment ([https://modis.gsfc.nasa.gov/data/](https://academy.climatematch.io/courses/2023-computational-tools-for-climate-science)). 

Before running the code please read : https://amanbagrecha.github.io/post/rs_gis/download-modis-data-using-cmr-api-in-python/ 


In [ ]:
#Import the libraries

from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler
from typing_extensions import Literal

In [ ]:
#download_modis.py
username = ""  # Update this line with your username 
password = ""  # Update this line with your password
# 1) connect to earthdata
session = ModisSession(username=username, password=password)

# 2) Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)
collections = collection_client.query(short_name="", version="") # Collection short name + version
# Query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

# 3) Filter the selected granules via spatial and temporal parameters
Israel_bbox = [] # format [x_min, y_min, x_max, y_max] # add min lon,min lat, max lon and max_lat input your preferred location
Israel_granules = granule_client.query(start_date="", end_date="", bounding_box=Israel_bbox)   #choose the start and end dates for the year-month-day you prefer

# 4) Download the granules
GranuleHandler.download_from_granules(Israel_granules, session, threads=-1) 
#NB the file format will be downloaded in hdf 


The following links could help students in preprocessing the MODIS dataset  
- https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/modis-data-in-python/

- http://www.hdfeos.org/zoo/LAADS/MYD08_D3.A2009001.006.2015088160849.hdf.py

- https://www.moonbooks.org/Articles/How-to-read-a-MODIS-HDF-file-using-python-/

However these were what i adopted from the code and how students could visualise the data 



In [ ]:
# from pyhdf.SD import SD, SDC

import numpy as np

file_name = 'D:/modis/modis.hdf'  #path + downloaded modis data

file = SD(file_name, SDC.READ)
file.info()
datasets_dic = file.datasets()
datasets_dic = file.datasets()
print(datasets_dic)
for idx,sds in enumerate(datasets_dic.keys()):
    print( idx,sds )
  sds_obj = file.select('Npp_500m') # select the data variable 

data = sds_obj.get() # get sds data

print( data )
 import pprint

 pprint.pprint( sds_obj.attributes() )
  for key, value in sds_obj.attributes().items():
      print( key, value )
      if key == 'add_offset':
          add_offset = value  
      if key == 'scale_factor':
          scale_factor = value
 data = (data - add_offset) * scale_factor
 data

# Plot the data
plt.imshow(data, cmap='jet')
plt.colorbar()
plt.title('Npp_500m')
plt.show()


The following papers could assist in processing of MODIS dataset: 
- https://www.mdpi.com/2072-4292/8/7/554 

- https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1430913

- https://www.mdpi.com/2072-4292/6/6/5368

##**MERRA-2 (Modern-Era Retrospective analysis for Research and Applications, Version 2)**

Further background on the dataset could be found at https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/ . 

In [ ]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

ModuleNotFoundError: ignored

- https://daac.gsfc.nasa.gov/information/howto?title=How%20to%20Access%20MERRA-2%20Data%20using%20OPeNDAP%20with%20Python3%20and%20Calculate%20Daily%2FWeekly%2FMonthly%20Statistics%20from%20Hourly%20Data%20

- https://github.com/emilylaiken/merradownload

- https://github.com/Open-Power-System-Data/weather_data/blob/master/download_merra2.ipynb

- https://daac.gsfc.nasa.gov/information/howto?title=How%20to%20remotely%20access%20MERRA-2%20with%20Python3%20and%20calculate%20monthly%20average%20surface%20PM2.5%20for%20world%20countries

- http://localhost:8888/notebooks/Downloads/How_to_read_and_plot_NetCDF_MERRA-2_data_in_Python%20(1).ipynb


#**Further reading**
- Zhao, X., Wu, T., Wang, S., Liu, K., Yang,  J. Cropland abandonment mapping at sub-pixel scales using crop phenological information and MODIS time-series images, Computers and Electronics in Agriculture, Volume 208,
2023,107763, ISSN 0168-1699,https://doi.org/10.1016/j.compag.2023.107763

- Shani Rohatyn et al.,  Limited climate change mitigation potential through forestation of the vast dryland regions.Science377,1436-1439(2022).DOI:10.1126/science.abm9684

- Hu, Y., Hou, M., Zhao, C., Zhen, X., Yao, L., Xu, Y. Human-induced changes of surface albedo in Northern China from 1992-2012, International Journal of Applied Earth Observation and Geoinformation, Volume 79, 2019, Pages 184-191, ISSN 1569-8432, https://doi.org/10.1016/j.jag.2019.03.018

- Duveiller, G., Hooker, J. & Cescatti, A. The mark of vegetation change on Earth’s surface energy balance. Nat Commun 9, 679 (2018). https://doi.org/10.1038/s41467-017-02810-8

- Yin, H., Brandão, A., Buchner, J., Helmers, D., Iuliano, B.G., Kimambo, N.E.,  Lewińska, K.E., Razenkova, E., Rizayeva, A., Rogova, N., Spawn, S.A., Xie, Y., Radeloff, V.C. Monitoring cropland abandonment with Landsat time series, Remote Sensing of Environment, Volume 246, 2020, 111873, ISSN 0034-4257,https://doi.org/10.1016/j.rse.2020.111873

- Gupta, P., Verma, S., Bhatla, R.,Chandel, A. S., Singh, J., & Payra, S.(2020). Validation of surfacetemperature derived from MERRA‐2Reanalysis against IMD gridded data setover India.Earth and Space Science,7,e2019EA000910. https://doi.org/10.1029/2019EA000910

- Cao, Y., S. Liang, X. Chen, and T. He (2015) Assessment of Sea Ice Albedo Radiative Forcing and Feedback over the Northern Hemisphere from 1982 to 2009 Using Satellite and Reanalysis Data. J. Climate, 28, 1248–1259, https://doi.org/10.1175/JCLI-D-14-00389.1.

- Westberg, D. J., P. Stackhouse, D. B. Crawley, J. M. Hoell, W. S. Chandler, and T. Zhang (2013), An Analysis of NASA's MERRA Meteorological Data to Supplement Observational Data for Calculation of Climatic Design Conditions, ASHRAE Transactions, 119, 210-221. 
https://www.researchgate.net/profile/Drury-Crawley/publication/262069995_An_Analysis_of_NASA's_MERRA_Meteorological_Data_to_Supplement_Observational_Data_for_Calculation_of_Climatic_Design_Conditions/links/5465225f0cf2052b509f2cc0/An-Analysis-of-NASAs-MERRA-Meteorological-Data-to-Supplement-Observational-Data-for-Calculation-of-Climatic-Design-Conditions.pdf